In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from keras.datasets import mnist
import keras
from keras.layers import *
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

import numpy as np
import matplotlib.pyplot as plt


In [ ]:
(X_train,_),(_,_)=mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
X_train.shape

(60000, 28, 28)

In [ ]:
X_train=(X_train-127.5)/127.5

print(X_train.min())
print(X_train.max())

-1.0
1.0


In [ ]:
TOTAL_EPOCHS =50
BATCH_SIZE=256
HALF_BATCH =128
NO_OF_BATCH=int(X_train.shape[0]/BATCH_SIZE)
NOISE_DIM=100
adam=Adam(lr=2e-4,beta_1=0.5)

In [ ]:
#Generator Model

generator=Sequential()
generator.add(Dense(units=7*7*128,input_shape=(NOISE_DIM,)))
generator.add(Reshape((7,7,128)))
generator.add(LeakyReLU(0.2))
generator.add(BatchNormalization())

#(7,7,128)->(14,14,64)

generator.add(Conv2DTranspose(64,(3,3),strides=(2,2),padding='same'))
generator.add(LeakyReLU(0.2))
generator.add(BatchNormalization())

#14,14,64 ->28,28,1

generator.add(Conv2DTranspose(1,(3,3),strides=(2,2),padding='same',activation='tanh'))
generator.compile(loss=keras.losses.binary_crossentropy,optimizer=adam)

generator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6272)              633472    
                                                                 
 reshape (Reshape)           (None, 7, 7, 128)         0         
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 7, 7, 128)         0         
                                                                 
 batch_normalization (BatchN  (None, 7, 7, 128)        512       
 ormalization)                                                   
                                                                 
 conv2d_transpose (Conv2DTra  (None, 14, 14, 64)       73792     
 nspose)                                                         
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 14, 14, 64)        0

In [ ]:
#Discriminator Model   ->Downsampling

#28,28,2 -> 14,14,64

discriminator=Sequential()
discriminator.add(Conv2D(64,kernel_size=(3,3),strides=(2,2),padding='same',input_shape=(28,28,1)))
discriminator.add(LeakyReLU(0.2))

#14,14,64 -> 7,7,128
discriminator.add(Conv2D(128,kernel_size=(3,3),strides=(2,2),padding='same'))
discriminator.add(LeakyReLU(0.2))

#7,7,128 ->6272
discriminator.add(Flatten())
discriminator.add(Dense(100))
discriminator.add(LeakyReLU(0.2))

discriminator.add(Dense(1,activation='sigmoid'))
discriminator.compile(loss=keras.losses.binary_crossentropy,optimizer=adam)
discriminator.summary()



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 14, 14, 64)        640       
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 14, 14, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 128)         73856     
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 7, 7, 128)         0         
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense_1 (Dense)             (None, 100)               627300    
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 100)              

In [ ]:
#COMBINED MODEL

discriminator.trainable=False
gan_input=Input(shape=(NOISE_DIM, ))
generated_img=generator(gan_input )
gan_output=discriminator(generated_img)

#Functional API
model=Model(gan_input,gan_output)
model.compile(loss=keras.losses.binary_crossentropy,optimizer=adam)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential (Sequential)     (None, 28, 28, 1)         708609    
                                                                 
 sequential_1 (Sequential)   (None, 1)                 701897    
                                                                 
Total params: 1,410,506
Trainable params: 708,225
Non-trainable params: 702,281
_________________________________________________________________


In [ ]:
X_train=X_train.reshape(-1,28,28,1)
X_train.shape

(60000, 28, 28, 1)

In [ ]:
def display_images(samples=25):
  noise=np.random.normal(0,1,size=(samples,NOISE_DIM))
  generated_img=generator.predict(noise)
  plt.figure(figsize=(10,10))
  for i in range(samples):
    plt.subplot(5,5,i+1)
    plt.imshow(generated_img[i].reshape(28,28), cmap="binary")
    plt.axis('off')
  plt.show()

In [ ]:
#Training Loop
d_losses=[]
g_losses=[]
for epoch in range(TOTAL_EPOCHS):
  epoch_d_loss=0.0
  epoch_g_loss=0.0

  #Mini batch gradient descent
  for step in range(NO_OF_BATCH):
    #1Training Descriminator
    discriminator.trainable=True

    #get the real data
    idx=np.random.randint(0,60000,HALF_BATCH)
    real_imgs = X_train[idx]

    #get fake data
    noise=np.random.normal(0,1,size=(HALF_BATCH,NOISE_DIM))
    fake_imgs=generator.predict(noise)

    #labels
    real_y=np.ones((HALF_BATCH,1))*0.9
    fake_y=np.zeros((HALF_BATCH,1))

    #Now, train D
    d_loss_real=discriminator.train_on_batch(real_imgs,real_y)
    d_loss_fake=discriminator.train_on_batch(fake_imgs,fake_y)

    d_loss=0.5*d_loss_real + 0.5*d_loss_fake

    epoch_d_loss += d_loss

    #2 TRAIN GENERATOR
    discriminator.trainable=False

    noise=np.random.normal(0,1,size=(BATCH_SIZE,NOISE_DIM))
    ground_truth_y=np.ones((BATCH_SIZE,1))
    g_loss=model.train_on_batch(noise,ground_truth_y)
    epoch_g_loss +=g_loss


    print(f"Epoch{epoch+1},Disc loss{epoch_d_loss/NO_OF_BATCH},Generator loss{epoch_d_loss/NO_OF_BATCH}")
    d_losses.append(epoch_d_loss/NO_OF_BATCH)
    g_losses.append(epoch_g_loss/NO_OF_BATCH)

    if(epoch+1)%10==0:
      generator.save("generator.h5")
      display_images()
